# LunAPI : introduction

Links to notebooks in this repository: [Index](./00_overview.ipynb) | [Luna tutorial](./tutorial.ipynb) | 
[Individuals](./01_indivs.ipynb) | [Projects](./02_projects.ipynb) | [Staging](./03_staging.ipynb) | [Models](./04_models.ipynb) | [Advanced](./98_advanced.ipynb) | [Reference](./99_reference.ipynb)

---

_v0.0.7(alpha) - 1-May-2024_

These pages give an introduction to the Python interface to [Luna](http://zzz.bwh.harvard.edu/luna).  _Please note that in this initial posting, some components are still place-holders: these will be updated over the coming weeks._   This notebook contains the following small sections:

 - Some notes to how best to [install LunAPI](#get-lunapi)
 - List of the current [tutorial notebooks](#tutorial-notebooks)
 - List of the current [reference notebooks](#reference-notebooks)
 - Some [first steps to get started](#first-steps) with _LunAPI_

These pages only contain information on _LunAPI_, the Python interface to Luna; for information on Luna and its commands, see the original [Luna documentation pages](http://zzz.bwh.harvard.edu/luna).  For general information on Python, JupyterLab or Docker, please see [this extensive and queryable collection of documents](http://www.google.com).

The `lunapi` package can be run in any Python environment.  However, for these notebooks, we assume that the [`remnrem/lunapi`](https://hub.docker.com/r/remnrem/lunapi) Docker image is being used (this only impacts the locations of the data and other resource files, which are bundled in the above image).

<a id='get-lunapi'></a>
## Obtaining LunaAPI

Currently, the best way to try the `lunapi` package is to obtain the Docker image.  Follow these instructions in the [README.md](README.md).  Platform-specific binary wheels for macOS and x86_64 Linux platforms are present in PyPI and can be installed with `pip install lunapi`.  If your particular platform or Python installation is not supported by a binary wheel, please use this Docker image for now.

> Note that the terms _LunAPI_ and `lunapi` are used interchangeably throughout these notes.


<a id='tutorial-notebooks'></a>
## Tutorial Notebooks

 - [Luna tutorial](./tutorial.ipynb): the original command-line [Luna tutorial](https://zzz.bwh.harvard.edu/tut/tut1) using `lunapi`

 - [Individuals](./01_indivs.ipynb): more information on working with a single individual

 - [Projects](./02_projects.ipynb): more information on working with multiple individuals

 - [Staging](./03_staging.ipynb): automated staging and stage evaluation

 - [Models](./04_models.ipynb): using PREDICT models

<a id='reference-notebooks'></a>
## Reference Notebooks

 - [More advanced topics](./98_advanced.ipynb): material to give deeper orientation on how the package is structured, along with various tips on usage

 - [Reference](./99_reference.ipynb): a complete reference of the `lunapi` Python package

<a id='first-steps'></a>
## First steps

Start by importing the `lunapi` library; in these notebooks by convention we use the alias `lp`:

In [1]:
import lunapi as lp

The second necessary step is to initiate a Luna _project_ using `proj()`.  Each Python session can contain no more than a single _project_ (which is an object of type `proj`). By convention, we'll call _projects_ `proj` in these notebooks, but you can use any label.

In [2]:
proj = lp.proj()

initiated lunapi v0.0.7 <lunapi.lunapi0.luna object at 0x28f34af70> 



_Projects_ control the creation of individual data _instances_, i.e. practically, an EDF and any associated annotation and meta-data for an individual.  The _project_ class also provides support for Luna [_sample-lists_](https://zzz.bwh.harvard.edu/luna/luna/args/#sample-lists) to facilitate multi-individual analyses.

To create an individual _instance_, use the _project_ class `inst()` function, which returns an `inst` object.  If not working with a sample-list, `inst()` requires an ID string as an argument, e.g. here `id1`.  By convention, we call all initial individual instances `p`:

In [3]:
p = proj.inst('id1')

At this point, `p` is just an empty placeholder.  We can directly attach an EDF file with `attach_edf()`.  For example, here we use one of the NSRR tutorial EDFs, which in the `remnrem/lunapi` Docker image are located at `/tutorial/edfs/`.  We first check these files are present:

In [4]:
%%sh
ls tutorial/edfs

learn-nsrr01.edf
learn-nsrr01.xml
learn-nsrr02.edf
learn-nsrr02.xml
learn-nsrr03.edf
learn-nsrr03.xml


That is, there are three EDFs, each with a corresponding XML annotation file.  Next, we attach `learn-nsrr02.edf` to the instance `p`. This will 1) print some logging information to the console, and 2) return `True` if it successfully completes:

In [5]:
p.attach_edf( 'tutorial/edfs/learn-nsrr02.edf' )

___________________________________________________________________
Processing: id1 | tutorial/edfs/learn-nsrr02.edf
 duration 09.57.30, 35850s | time 21.18.06 - 07.15.36 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT


True

As described in subsequent notebooks, `lunapi` supplies a range of functions for working with _projects_ and _instances_.  For example, the `stat()` function returns a Pandas dataframe with some basic information about that instance (e.g. the number of channels, `nc`, etc, see the [reference page](./99_reference.ipynb) for full details):

In [6]:
p.stat()

,Value
annotation_files,
duration,09.57.30.000
edf_file,tutorial/edfs/learn-nsrr02.edf
id,id1
na,0
ns,14
nt,14
state,1


The `desc()` function gives some more expanded statistics:

In [7]:
p.desc()

,ID,Gapped,Date,Start(hms),Stop(hms),Dur(hms),Dur(s),# sigs,# annots,Signals
1,id1,N,01.01.85,21.18.06,07.15.36,09:57:30,35850,14/14,0,SaO2[1] PR[1] EEG_sec[125] ECG[250] EMG[125] EOG_L[50] EOG_R[50] EEG[125] AIRFLOW[10] THOR_RES[10] ABDO_RES[10] POSITION[1] LIGHT[1] OX_STAT[1]


One major _instance_ member function is `eval()` (and its variant `proc()`, which just differs in how it returns results to Python).  The `eval()` function executes arbitrary Luna commands on an attached EDF.  For example, here we will run the [`HEADERS`](https://zzz.bwh.harvard.edu/luna/ref/summaries/#headers) command, which returns basic information on (guess what) the EDF headers:

In [8]:
p.eval( 'HEADERS' )

 ..................................................................
 CMD #1: HEADERS
   options: sig=*


,Command,Strata
0,HEADERS,BL
1,HEADERS,CH


Any console/log output is displayed by default.  You can silence this logging information by running `proj.silence()` (see [references](./99_reference.ipynb) for details).

The `eval()` function also returns a list of the table(s) generated by the executed command(s); the actual results are stored within the instance object referenced by `p`.  That is, this function alters the _instance_ referenced by `p`.  You can access these single result tables with `table()`, which takes two arguments: the command and strata names (i.e. corresponding to a row from the table above).

In [9]:
p.table( 'HEADERS' , 'CH' )

,ID,CH,DMAX,DMIN,PDIM,PMAX,PMIN,POS,SENS,SR,TRANS,TYPE
0,id1,ABDO_RES,127,-128,.,-1.0,1.0,11,-0.007843,10.0,.,EFFORT
1,id1,AIRFLOW,127,-128,.,-1.0,1.0,9,-0.007843,10.0,.,AIRFLOW
2,id1,ECG,127,-128,mV,1.25,-1.25,4,0.009804,250.0,.,ECG
3,id1,EEG,127,-128,uV,125.0,-125.0,8,0.980392,125.0,.,EEG
4,id1,EEG_sec,127,-128,uV,125.0,-125.0,3,0.980392,125.0,.,EEG
5,id1,EMG,127,-128,uV,31.5,-31.5,5,0.247059,125.0,.,EMG
6,id1,EOG_L,127,-128,uV,125.0,-125.0,6,0.980392,50.0,.,EOG
7,id1,EOG_R,127,-128,uV,125.0,-125.0,7,0.980392,50.0,.,EOG
8,id1,LIGHT,1,0,.,1.0,0.0,13,1.0,1.0,.,LIGHT
9,id1,OX_STAT,3,0,.,3.0,0.0,14,1.0,1.0,.,OXYGEN


Any sequence of Luna commands can be passed to `eval()` or `proc()`, making these the primary modes for using `lunapi`. 

There are a number of other ways to interact with _projects_ and attached _instances_, including working with samples instead of one individual at a time (i.e. via _sample-lists_), or using lower-level functions to communicate with the underlying C/C++ library more directly.  The package also contains functions to interact with the raw signals and annotations directly in Python.  All of these things are described in subsequent notebooks in this repository.

That's it for this first pass.  You can now proceed to the next notebook, which gives a deeper set of examples for [working with individual-level signal and annotation data](./01_indivs.ipynb).